# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 17 2023 9:56AM,261177,10,CTF0011723,"Citieffe, Inc.",Released
1,Apr 17 2023 9:56AM,261177,10,CTF0011724,"Citieffe, Inc.",Released
2,Apr 17 2023 9:56AM,261177,10,CTF0011725,"Citieffe, Inc.",Released
3,Apr 17 2023 9:48AM,261175,10,EYE43040,Eye Pharma Inc,Released
4,Apr 17 2023 9:48AM,261175,10,EYE43041,Eye Pharma Inc,Released
5,Apr 17 2023 9:48AM,261175,10,EYE43042,Eye Pharma Inc,Released
6,Apr 17 2023 9:48AM,261175,10,EYE43043,Eye Pharma Inc,Released
7,Apr 17 2023 9:48AM,261175,10,EYE43044,Eye Pharma Inc,Released
8,Apr 17 2023 9:47AM,261173,10,9379831,Eye Pharma Inc,Released
9,Apr 17 2023 9:38AM,261172,19,TR-RA-04182023,"GCH Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,261171,Released,1
24,261172,Released,1
25,261173,Released,1
26,261175,Released,5
27,261177,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261171,NaN,1.0
261172,NaN,1.0
261173,NaN,1.0
261175,NaN,5.0
261177,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261142,0.0,1.0
261145,11.0,3.0
261146,0.0,1.0
261147,0.0,1.0
261149,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261142,0,1
261145,11,3
261146,0,1
261147,0,1
261149,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261142,0,1
1,261145,11,3
2,261146,0,1
3,261147,0,1
4,261149,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261142,,1
1,261145,11,3
2,261146,,1
3,261147,,1
4,261149,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 17 2023 9:56AM,261177,10,"Citieffe, Inc."
3,Apr 17 2023 9:48AM,261175,10,Eye Pharma Inc
8,Apr 17 2023 9:47AM,261173,10,Eye Pharma Inc
9,Apr 17 2023 9:38AM,261172,19,"GCH Granules USA, Inc."
10,Apr 17 2023 9:38AM,261171,21,"NBTY Global, Inc."
11,Apr 17 2023 9:36AM,261170,10,"CLINUVEL, Inc."
12,Apr 17 2023 9:35AM,261168,10,"Methapharm, Inc."
52,Apr 17 2023 9:35AM,261169,15,"Brookfield Pharmaceuticals, LLC"
73,Apr 17 2023 9:33AM,261165,10,"Methapharm, Inc."
74,Apr 17 2023 9:27AM,261164,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 17 2023 9:56AM,261177,10,"Citieffe, Inc.",,3
1,Apr 17 2023 9:48AM,261175,10,Eye Pharma Inc,,5
2,Apr 17 2023 9:47AM,261173,10,Eye Pharma Inc,,1
3,Apr 17 2023 9:38AM,261172,19,"GCH Granules USA, Inc.",,1
4,Apr 17 2023 9:38AM,261171,21,"NBTY Global, Inc.",,1
5,Apr 17 2023 9:36AM,261170,10,"CLINUVEL, Inc.",,1
6,Apr 17 2023 9:35AM,261168,10,"Methapharm, Inc.",,40
7,Apr 17 2023 9:35AM,261169,15,"Brookfield Pharmaceuticals, LLC",13,8
8,Apr 17 2023 9:33AM,261165,10,"Methapharm, Inc.",,1
9,Apr 17 2023 9:27AM,261164,10,"Methapharm, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 17 2023 9:56AM,261177,10,"Citieffe, Inc.",3,
1,Apr 17 2023 9:48AM,261175,10,Eye Pharma Inc,5,
2,Apr 17 2023 9:47AM,261173,10,Eye Pharma Inc,1,
3,Apr 17 2023 9:38AM,261172,19,"GCH Granules USA, Inc.",1,
4,Apr 17 2023 9:38AM,261171,21,"NBTY Global, Inc.",1,
5,Apr 17 2023 9:36AM,261170,10,"CLINUVEL, Inc.",1,
6,Apr 17 2023 9:35AM,261168,10,"Methapharm, Inc.",40,
7,Apr 17 2023 9:35AM,261169,15,"Brookfield Pharmaceuticals, LLC",8,13
8,Apr 17 2023 9:33AM,261165,10,"Methapharm, Inc.",1,
9,Apr 17 2023 9:27AM,261164,10,"Methapharm, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 17 2023 9:56AM,261177,10,"Citieffe, Inc.",3,
1,Apr 17 2023 9:48AM,261175,10,Eye Pharma Inc,5,
2,Apr 17 2023 9:47AM,261173,10,Eye Pharma Inc,1,
3,Apr 17 2023 9:38AM,261172,19,"GCH Granules USA, Inc.",1,
4,Apr 17 2023 9:38AM,261171,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 17 2023 9:56AM,261177,10,"Citieffe, Inc.",3.0,NaN
1,Apr 17 2023 9:48AM,261175,10,Eye Pharma Inc,5.0,NaN
2,Apr 17 2023 9:47AM,261173,10,Eye Pharma Inc,1.0,NaN
3,Apr 17 2023 9:38AM,261172,19,"GCH Granules USA, Inc.",1.0,NaN
4,Apr 17 2023 9:38AM,261171,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 17 2023 9:56AM,261177,10,"Citieffe, Inc.",3.0,0.0
1,Apr 17 2023 9:48AM,261175,10,Eye Pharma Inc,5.0,0.0
2,Apr 17 2023 9:47AM,261173,10,Eye Pharma Inc,1.0,0.0
3,Apr 17 2023 9:38AM,261172,19,"GCH Granules USA, Inc.",1.0,0.0
4,Apr 17 2023 9:38AM,261171,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3133987,116.0,3.0
15,261169,8.0,13.0
16,261153,0.0,2.0
18,522310,1.0,1.0
19,1305772,7.0,11.0
20,261142,1.0,0.0
21,261171,1.0,0.0
22,522310,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3133987,116.0,3.0
1,15,261169,8.0,13.0
2,16,261153,0.0,2.0
3,18,522310,1.0,1.0
4,19,1305772,7.0,11.0
5,20,261142,1.0,0.0
6,21,261171,1.0,0.0
7,22,522310,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,116.0,3.0
1,15,8.0,13.0
2,16,0.0,2.0
3,18,1.0,1.0
4,19,7.0,11.0
5,20,1.0,0.0
6,21,1.0,0.0
7,22,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,116.0
1,15,Released,8.0
2,16,Released,0.0
3,18,Released,1.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Executing,3.0,13.0,2.0,1.0,11.0,0.0,0.0,1.0
Released,116.0,8.0,0.0,1.0,7.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Executing,3.0,13.0,2.0,1.0,11.0,0.0,0.0,1.0
1,Released,116.0,8.0,0.0,1.0,7.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Executing,3.0,13.0,2.0,1.0,11.0,0.0,0.0,1.0
1,Released,116.0,8.0,0.0,1.0,7.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()